# Evaluate the FS algo

Take a time window of prices and decide if SELL, HOLD, BUY, untill the next transaction time.
Iterate through whole history of one stock

In [6]:
using EO, Plots, Lasso

## Stock picking

In [7]:
function generate_predictions(x_max, price, training_width, prediction_width, timestep, expert_cnt)

    ps = Vector{Vector{Vector{Float64}}}()

    iter = 1
    training_i = 2:training_width+1
    while training_i[end] + prediction_width < x_max
        x = float.(collect(training_i))                                         # training set
        xt = float.(collect(training_i[1]:(training_i[end]+prediction_width)))  # testing set

        # define training
        basis_functions = [+, -, *, /, sin, cos]
        basis_variables = Vector{Any}([:var1])
        pop_size = 1

        objective_function  = enclose_arguments(EO.f_function_diff_squared, x, float.(price[training_i]))
        initialization      = enclose_noargs(EO.one_expression_initialization, pop_size, objective_function, basis_functions, basis_variables)
        selection           = EO.s_identity
        crossover           = identity
        mutation            = enclose_arguments(EO.subtree_mutation!, basis_functions, basis_variables)
        replacement         = EO.enclose_replacement(r_keep_best_n, 1)
        termination         = enclose_argument(iteration_termination, pop_size*100)
        
        # train expert models
        predictions = Vector{Vector{Float64}}()
        while length(predictions) < expert_cnt
            print("\rtraining expert $(length(predictions)+1) at iter $iter")
            try         # optimization can diverge
                # train a model

                @time solution = solvink_hart(objective_function, initialization, selection, crossover, mutation, replacement, termination)

                model = EO.Expr_parser(solution);
                prediction = model.(xt)
                push!(predictions, prediction)
            catch
                display("failed")
                continue
            end
        end
        iter += 1
        # evaluate the predictions
        push!(ps, predictions)

        #training_i .+= timestep
        training_i = (training_i[1]+timestep):(training_i[end]+timestep)
    end
    return ps
end

generate_predictions (generic function with 1 method)

In [8]:
#= using CSV, DataFrames, FileIO

set = "Hanz"
names = ["3M", "Apple", "Coca-Cola", "Exxon Mobil", "Ford Motor", "Intel", "Microsoft", "NVIDIA", "Pfizer"]
#set = "SAT"
#names = ["EA", "K"]
name = names[1]
if set == "SAT"
    data = CSV.read("../trading_data/archive(4)/sp500_stocks.csv", DataFrame)
    price = filter("Symbol" => x->x == name, data).Open
else
    data = CSV.read("../trading_data/whole_dataset/"*name*" Stock Price History.csv", DataFrame)
    price = reverse(data.Price)
end

x = collect(1:length(price))

training_width = 100
prediction_width = 10
timestep = 10
expert_cnt = 5

#res = generate_predictions(length(price), price, training_width, prediction_width, timestep, expert_cnt);
res = generate_predictions(length(price), price, training_width, prediction_width, timestep, expert_cnt);
# save for reuse
save(joinpath(@__DIR__, "evaluations", "LS", name, "predictions.jld2"), "res", [res, price]) =#

In [9]:
using EO, Plots, Lasso, CSV, DataFrames, FileIO

set = "Hanz"
names = [#= "3M",  =#"Apple", "Coca-Cola", "Exxon Mobil", "Ford Motor", "Intel", "Microsoft", "NVIDIA", "Pfizer"]
#set = "SAT"
#names = ["EA", "K"]

for name in names

    if set == "SAT"
        data = CSV.read("../trading_data/archive(4)/sp500_stocks.csv", DataFrame)
        price = filter("Symbol" => x->x == name, data).Open
    else
        data = CSV.read("../trading_data/whole_dataset/"*name*" Stock Price History.csv", DataFrame)
        price = reverse(data.Price)
    end
    
    x = collect(1:length(price))

    training_width = 100
    prediction_width = 10
    timestep = 10
    expert_cnt = 10
    
    res = generate_predictions(length(price), price, training_width, prediction_width, timestep, expert_cnt);
    # save for reuse
    save(joinpath(@__DIR__, "evaluations", "LS", name, "predictions.jld2"), "res", [res, price])
end